# Preliminaries

In [3]:
import pandas as pd  #importing all the important packages
import numpy as np
import matplotlib.pyplot as plt #currently not used
import seaborn as sns #currently not used
import pickle

Import the table containing nationaldex pokemon data up until generation 8.

In [4]:
pickle_in = open('pokemon.pickle','rb')
pokemon = pickle.load(pickle_in)
pokemon.columns = pokemon.columns.str.upper().str.replace('_', '') 
pokemon['TYPE2'] = pokemon['TYPE2'].replace(to_replace=[None], value=np.nan, inplace=True)
pokemon['TYPE2'].fillna(pokemon['TYPE1'], inplace=True)
pokemon = pokemon.set_index('NAME')
pokemon=pokemon.drop(['ID'],axis=1)

Import the table containing all moves data up until generation 8.

In [5]:
pickle_in = open('moves.pickle','rb')
moves = pickle.load(pickle_in)

We can import functions from other .ipynb files using:

In [6]:
from ipynb.fs.defs.Functions import type_weakness,dmg,base_stats_at_lvl,phys_spec_ratio,get_moves